# When worlds collide even more: constrainted multi-objective optimization under uncertainty

In the previous optimization notebook (["freyberg_mou_1](../part2_09_mou/freyberg_mou_1.ipynb)), we saw how we can use PESTPP-MOU to do some pretty killer decision support with multi-objective optimization.  It was awesome...Now, lets see how stacks, risk, and reliability come into play here.  

Generally the concept is the same as with chance constraints that we saw in the second PESTPP-OPT notebook.  However, PESTPP-MOU extends the "chance" concept to also include objectives derived from model outputs. Like model-based constraints and predictions/forecasts, these objectives are also subject to uncertainty, in as much as these objectives depend on model inputs (i.e. parameters) that are uncertain.  

But things get complicated now.  In PESTPP-OPT, we had a single "solution" (i.e. set of decision variables) but PESTPP-MOU works with populations of individual decision variable vectors.  Does this mean that we need to evaluate the stack at each individual?! And what about each generation?!  OMG - that is an insane number of runs!  Luckily PESTPP-MOU lets you "reuse" the stack evaluation results across individuals and across generations.  The former is controlled by the "opt_chance_points" argument, while the latter is controlled by "opt_recalc_chance_every" argument.  Again, the choice for how to evaluate chances must be a tradeoff between computational burden and coupling expectations (and how that coupling might effect the pareto-optimal set).  

Ok, so how is this sharing accomplished?  Well, its a matter of finding the "nearest" point in decision variable space where the stack has been evaluated for each individual that needs chance info, and then differencing the mean value of the constraint/objective stack distribution and the corresponding value for the constraint/objective value at the individual.  That is just book-keeping - easy as!

#### But, wait! there is more.

What if the risk/reliability value for decision makers is not known and therefore cannot be provided as an input to PESTPP-MOU?  Or what if you aren't sure if a given risk/reliability value will cause infeasibility?  What can we do about this?  

Well, what if we treat risk as just another objective that we want to maximize (that is prefer more risk averse solutions)?.  In this case, PESTPP-MOU will simply look for pareto-optimal combinations of the objectives (which also includes varying risk values).  WAT?!  Yeah, that's right!  You can treat risk as an objective and then the results of a PESTPP-MOU run will include a range of risk stances that can be filters/sliced/diced however you want. BOOM!


### Admin

Start off with the usual loading of dependencies and preparing model and PEST files. We will be continuing to work with the modified-Freyberg model (see ["freyberg intro to model"](../part0_02_intro_to_freyberg_model/intro_freyberg_model.ipynb) notebook), and the high-dimensional PEST dataset prepared in the ["freyberg pstfrom pest setup"](../part2_01_pstfrom_pest_setup/freyberg_pstfrom_pest_setup.ipynb)" and ["observation and weights"](../part2_02_obs_and_weights/freyberg_obs_and_weights.ipynb) notebooks. 

For the purposes of this notebook, you do not require familiarity with previous notebooks (but it helps...). 

Simply run the next few cells by pressing `shift+enter`.

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import numpy as np
import pandas as pd
font = {'size'   : 15}
import matplotlib
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt;
import shutil
import psutil

import sys 
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd

This notebook builds on having run both the first [PESTPP-MOU notebook](../part2_09_mou/freyberg_mou_1.ipynb) and the first [PESTPP-IES notebook](../part2_06_ies/freyberg_ies_1_basics.ipynb)...

In [ ]:
# specify the temporary working folder
t_d = os.path.join('freyberg6_template_chance')
if os.path.exists(t_d):
    shutil.rmtree(t_d)

org_t_d = os.path.join("freyberg6_template")
if not os.path.exists(org_t_d):
    raise Exception("you need to run the '/part2_08_opt/freyberg_mou_1.ipynb' notebook")

shutil.copytree(org_t_d,t_d)

In [ ]:
pst_path = os.path.join(t_d, 'freyberg_mf6.pst')

In [ ]:
pst = pyemu.Pst(pst_path)

## Stacks

Just the same code as we saw with the second [PESTPP-OPT notebook](../part2_08_opt/freyberg_opt_2.ipynb) (hopefully you did that notebook already...)

In [ ]:
# check that the pestpp-ies directory exists and that the posterior parameter ensemble exists
ies_dir = os.path.join("..","part2_06_ies","master_ies_1")
if not os.path.exists(ies_dir):
    raise Exception("you need to run the 'part2_06_ies/freyberg_ies_1_basics.ipynb' notebook")

Get the parameter ensemble from the PESTPP-IES run:

In [ ]:
pe_files = [f for f in os.listdir(ies_dir) if f.endswith(".par.csv") and f.startswith("freyberg_mf6") and "mean" not in f]
pe_files.sort()
pe_files

Now load the parameter ensemble from the last iteration of PESTPP-IES:

In [ ]:
last_iter = max([int(i.split('.')[-3]) for i in pe_files if i.split('.')[-3].isdigit()])
last_iter_pe_file = [i for i in pe_files if i.split('.')[-3]==str(last_iter)][0]
last_iter_pe_file

In [ ]:
pe = pd.read_csv(os.path.join(ies_dir,last_iter_pe_file),index_col=0, low_memory=False)
pe.head()

In [ ]:
par = pst.parameter_data
par.loc[par.partrans=="fixed","partrans"] = "none"

# identify the well pumping rate parameters
wpar = par.loc[par.parnme.str.contains("wel") & par.parnme.str.contains("cn"),"parnme"]
pe.loc[:,wpar.values] = 1.0

Record the parameter stack to external file:

In [ ]:
pe.to_csv(os.path.join(t_d,"par_stack.csv"))

### Risk as an objective

To treat risk as an objective, first we need to add a special parameter to the control file.  This parameter is not used to run the model, but for design reasons, we need to have it in a template file.  

So let's make a dummy template file for a parameter called "\_risk\_" - it has to be named just like this.  Once we have that template file, we can use the `pyemu.Pst` machinery to add that parameter to the pest interface and some bound and initial value info and add this parameter to the decision variable parameter group.  

We then need a prior information equation that involves only this "\_risk\_" parameter with group name of "greater_than"

In [ ]:
risk_tpl = os.path.join(t_d,"risk.dat.tpl")
with open(risk_tpl,'w') as f:
          f.write("ptf ~\n")
          f.write("risk ~ _risk_~ \n")
pst.add_parameters(risk_tpl,pst_path=".")
par = pst.parameter_data
par.loc["_risk_","partrans"] = "none"
par.loc["_risk_","parubnd"] = 0.999
par.loc["_risk_","parlbnd"] = 0.001
par.loc["_risk_","parval1"] = 0.001
par.loc["_risk_","pargp"] = "decvars"

Add the prior info equation:

In [ ]:
pst.add_pi_equation(["_risk_"],pilbl="_risk_",obs_group="greater_than")

In [ ]:
pst.prior_information

Now just a little more book keeping - we need to add the "\_risk\_" parameter to the initial decision variable population since all adjustable decision variables must be in this population file (we can also cheat and use the previous MOU run results from the last generation as the initial generation for this MOU run - this will help by starting us closer to the optimal pareto frontier)

In [ ]:
# read the initial dec var file we created in the previous tutorial
dv_file = os.path.join(t_d,"initial_dvpop.csv")

# lets see if the previous mou results are available:
mou1_m_d = os.path.join("master_mou_1")
if os.path.exists(mou1_m_d):
    files = [f for f in os.listdir(mou1_m_d) if f.endswith("dv_pop.csv") 
                     and "archive" not in f and len(f.split('.')) == 4]
    gen = [int(f.split(".")[1]) for f in files]
    idx = gen.index(max(gen))
    dv_file = os.path.join(mou1_m_d,files[idx])
print(dv_file)
df = pd.read_csv(dv_file,index_col=0)
# adda new column for the _risk_ parameter/decision variable
df.loc[:,"_risk_"] = np.random.uniform(0.001,0.999,df.shape[0])
#record it as external file again
df.to_csv(os.path.join(t_d,"initial_dvpop.csv"))

#### Some PESTPP-MOU Optional Variables

Now just a few additions to the optional pest++ args.  Of note here:  
 - we add "\_risk\_" to the objectives, we tell PESTPP-MOU we explicitly want to use risk as an objective, and 
 - we want to evaluate the stack at all individuals ("opt_chance_points" = "all") and then,
 - we want to reuse those stack evaluations across all remaining generations ("opt_recalc_chance_every" = 1000)

In [ ]:
pst.pestpp_options["mou_objectives"] += ",_risk_"
pst.pestpp_options["mou_risk_objective"] = True
pst.pestpp_options["opt_chance_points"] = "all"
pst.pestpp_options["opt_recalc_chance_every"] = 1000

We also want to:
 - make sure the parameter stack is read from "par_stack.csv";
 - specify our preferred risk-stance (in the case of risk-as-an-objective, we just need to make sure risk != 0.5 to activate the risk/reliability process)

In [ ]:
pst.pestpp_options["opt_par_stack"] = "par_stack.csv"
pst.pestpp_options["opt_risk"] = 0.95

In [ ]:
obs_org = pst.observation_data.copy()
obs = pst.observation_data
#obs.loc[obs.apply(lambda x: x.weight > 0 and "wel" in x.obsnme,axis=1),"weight"] = 0.0

In [ ]:
pst.control_data.noptmax = 50
pst.write(pst_path,version=2)


# Attention!

You must specify the number which is adequate for ***your*** machine! Make sure to assign an appropriate value for the following `num_workers` variable:

In [ ]:
num_workers = 15 # update according to your available resources!

Then specify the folder in which the PEST manager will run and record outcomes. It should be different from the `t_d` folder. 

In [ ]:
m_d = os.path.join('master_mou_2')
#if os.path.exists(m_d):
#    shutil.rmtree(m_d)

The following cell deploys the PEST agents and manager and then starts the run using `pestpp-opt`. Run it by pressing `shift+enter`.

If you wish to see the outputs in real-time, switch over to the terminal window (the one which you used to launch the `jupyter notebook`). There you should see `pestpp-opt`'s progress. 

If you open the tutorial folder, you should also see a bunch of new folders there named `worker_0`, `worker_1`, etc. These are the agent folders. `pyemu` will remove them when PEST finishes running.

This run should take a while to complete (depending on the number of workers and the speed of your machine). If you get an error, make sure that your firewall or antivirus software is not blocking `pestpp-opt` from communicating with the agents (this is a common problem!).

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-mou","freyberg_mf6.pst",num_workers=num_workers,worker_root=".",
                           master_dir=m_d)

## Post-processing

In the previous MOU tutorial we described the types of output files. Once again, we will inspect outcomes in the archive summary file. Note that this time, we also have a column for "\_risk_\"!

In [ ]:
df = pd.read_csv(os.path.join(m_d,"freyberg_mf6.pareto.archive.summary.csv"))
df.tail()

As in the last tutorial, filter for feasible and nondominated individuals:

In [ ]:
feas_front_df = df.loc[df.apply(lambda x: x.nsga2_front==1 and x.is_feasible==1,axis=1),:]

And then our function to make cool plots of all these things:

In [ ]:
def plot_pareto(feas_df,gen,min_risk=0.0,max_risk=1.0):
    """make a cross plot for a given generation.  Note the axes ranges are 
    sync'd across all generations so you can compare the plots more easily
    
    """
    objs = pst.pestpp_options["mou_objectives"].split(",")
    print(objs)
    df = feas_df.loc[feas_df.generation==gen,:]
    if "_risk_" in df.columns:
        df = df.loc[df._risk_>min_risk,:]
        df = df.loc[df._risk_<max_risk,:]
    elif "_risk_" in objs:
        objs.remove("_risk_")
    
    fig,axes = plt.subplots(len(objs),len(objs),figsize=(10,10))
    fig.suptitle("{0} solutions".format(df.shape[0]))
    onames = [o if "risk" in o else o.split("usecol:")[1].split('_')[0] for o in objs]
    for i in range(len(objs)):
        for j in range(i+1):
            if i == j:
                axes[i,j].hist(df.loc[:,objs[i]],alpha=0.5,facecolor="0.5") 
                axes[i,j].set_title(onames[i])
                axes[i,j].set_xlim(df.loc[:,objs[i]].min(),
                      df.loc[:,objs[i]].max())
                axes[i,j].set_yticks([])
                
            else:
                colors = "0.5"
                if "_risk_" in df.columns:
                    colors = df.loc[:,"_risk_"].values
                axes[j,i].scatter(df.loc[:,objs[i]],df.loc[:,objs[j]],marker=".",c=colors,vmax=1.0,vmin=0.0)
                
                axes[i,j].scatter(df.loc[:,objs[j]],df.loc[:,objs[i]],marker=".",c=colors,vmax=1.0,vmin=0.0)
                axes[j,i].set_xlim(df.loc[:,objs[i]].min(),
                      df.loc[:,objs[i]].max())
                axes[j,i].set_ylim(df.loc[:,objs[j]].min(),
                      df.loc[:,objs[j]].max())
                axes[j,i].set_title("{0} vs {1}".format(onames[i],onames[j]))
                
                axes[i,j].set_xlim(df.loc[:,objs[j]].min(),
                      df.loc[:,objs[j]].max())
                axes[i,j].set_ylim(df.loc[:,objs[i]].min(),
                      df.loc[:,objs[i]].max())
                axes[i,j].set_title("{0} vs {1}".format(onames[j],onames[i]))
    
    plt.tight_layout()
    return fig,axes

So first, let's see the pareto frontier among our three objectives in the final (best) generation. 

As in the previous tutorial, in the diagonal we see the histograms of both objectives - the distribution of their values in the final population. The off-diagonals are where things get interesting: These show the pareto frontier between extracting more groundwater and leaving groundwater for surface-water/groundwater exchange. (Note the negative values on the axes).

What is new here, is that we are also displaying the reliability/risk (colours). Bright/yellow denotes high reliability (low risk), and dark/purple low reliability (high risk). 

In [ ]:
fig,axes = plot_pareto(feas_front_df,feas_front_df.generation.max());
plt.show()

Just take a moment to let that soak in - wow!  So what we see is that the highly reliable solutions (yellow) are clustered around the lower values of water use (duh!) and also are clustered around the lower values of sw-gw exchange volume.  This should be intuitive to you, especially when we remember that all sources of water in to the domain are treated as uncertain.  So this means reliable solutions are going to be coping with less available water. 

How about we focus on highly reliable solutions:

In [ ]:
fig,axes = plot_pareto(feas_front_df,feas_front_df.generation.max(),0.95)

plt.show()

If we compare these results to the previous risk-neutral opt (see below also), we see that we are getting a lot __less__ groundwater extraction - this is the cost of uncertainty...if you want to be _certain_ that a "bad thing" won't happen, then you must make do with less.

How about we check the explicitly risk-neutral solutions (with a tolerance around risk = 0.5):

In [ ]:
fig,axes = plot_pareto(feas_front_df,feas_front_df.generation.max(),0.42,0.58)
plt.show()

The points are colored the same - make sure you understand where these are on the previous plot...not many solutions, feel free to play with the risk range...

Now let's compare these "risk neutral" solutions with the results from the previous MOU run (which was implicitly risk neutral). Following cells read in the results from the previous MOU tutorial:

In [ ]:
# get original results
df_org = pd.read_csv(os.path.join("master_mou_1","freyberg_mf6.pareto.archive.summary.csv"))
# filter for feasibility 
feas_front_df_org = df_org.loc[df_org.apply(lambda x: x.nsga2_front==1 and x.is_feasible==1,axis=1),:]

In [ ]:
fig,axes = plot_pareto(feas_front_df_org,feas_front_df_org.generation.max(),0.45,0.55);
plt.show()

Ok, so we see that the implicitly risk neutral (first) MOU run found many more solutions along the trade off between water use and sw-gw exchange.  This is because it didn't have the risk objective to deal with.  

This is an important consideration in multi-objective optimization in general:  the __more objectives__ you use, the __harder the algorithm has to work__ and the more generations that will be needed (i.e. more model runs).   But the current MOU run with risk as an objective is clearly in the same range of solutions and (if you squint) you can see that the current run also has the same directionality in the trade off.